## Vancouver Public Washrooms

A notebook for exporting the [Vancouver Public Washrooms dataset](https://opendata.vancouver.ca/explore/dataset/public-washrooms/information/) to GPX.

In [1]:
import requests
import gpxpy.gpx
from datetime import datetime

First, make a request to see how many records are in this dataset, logged in the `nhits` field.

In [2]:
url = 'https://opendata.vancouver.ca/api/records/1.0/search/'
res = requests.get(url, params={'dataset': 'public-washrooms'})
nrows = res.json()['nhits']
now = datetime.now()
print('As of {}-{}-{}, there are {} public restrooms listed in this dataset'.format(now.year, now.month, now.day, nrows))

As of 2020-12-29, there are 106 public restrooms listed in this dataset


## Fetching data
Now that I know how many records there are, I download them all in one request.

In [3]:
payload = {
    'dataset': 'public-washrooms',
    'rows': nrows,
    'facet': ['type', 'summer_hours', 'winter_hours', 'wheel_access', 'maintainer', 'geo_local_area'],
}
res = requests.get(url, params=payload)
data = dict(res.json())
records = data['records']

Upon inspecting the first three records, it seems all the salient information is in the `fields` dictionary.

In [4]:
records[0:3]

[{'datasetid': 'public-washrooms',
  'recordid': '793e7960e380aa663040ad6bc95f66c0cdc77236',
  'fields': {'maintainer': 'Parks & Recreation',
   'name': 'Adanac Park',
   'wheel_access': 'No',
   'primaryind': '1',
   'note': 'Caretaker on site',
   'summer_hours': 'Dawn to Dusk',
   'geom': {'type': 'Point',
    'coordinates': [-123.024071960319, 49.2758809677686]},
   'location': 'East side, fieldhouse',
   'address': '1025 Boundary Road',
   'winter_hours': 'Dawn to Dusk',
   'type': 'Public Toilet in Park',
   'geo_local_area': 'Hastings-Sunrise'},
  'record_timestamp': '2020-10-15T10:31:50.455000+00:00'},
 {'datasetid': 'public-washrooms',
  'recordid': '7a670401a7376b658683fbfabd6eba6855d20896',
  'fields': {'maintainer': 'Parks & Recreation',
   'name': 'Balaclava Park',
   'wheel_access': 'No',
   'primaryind': '3',
   'note': 'Caretaker on site',
   'summer_hours': 'Dawn to Dusk',
   'geom': {'type': 'Point',
    'coordinates': [-123.175460949361, 49.2452339943975]},
   'locat

## Convert to GPX

I'll convert and modify this JSON data into GPX format. I'll also modify some fields, such as combining many fields into a the `description`.

In [5]:
def format_description(fields):
    ''''''    
    out = ''
    out += 'A {}, '.format(fields['type'].lower())
    out += 'open from '
    if fields['summer_hours'] != fields['winter_hours']:
        out += '{} in the summer, but {} in the winter.'.format(fields['summer_hours'].lower(), fields['winter_hours'].lower())
    else:
        out += fields['summer_hours'].lower()
    
    if fields['wheel_access'] == 'Yes':
        out += ' with wheelchair access'
    else:
        out += ' with no wheelchair access'
    return out

gpx = gpxpy.gpx.GPX()
gpx.creator = 'Steve Kasica, kasica@alumni.ubc.ca'
gpx.name = 'Vancouver Public Washrooms'
gpx.description = 'Public washrooms in Vancouver'
desc_template = 'A {washroom_type} open.'
for r in records:
    wps = gpxpy.gpx.GPXWaypoint()
    wps.latitude = r['fields']['geom']['coordinates'][1]
    wps.longitude = r['fields']['geom']['coordinates'][0]
    wps.source = 'City of Vancouver'
    wps.time = datetime.strptime(r['record_timestamp'], '%Y-%m-%dT%H:%M:%S.%f+00:00')
    wps.type = 'Public washroom'
    wps.symbol = 'Restroom'
    wps.name = r['fields']['name'] + ' Washroom'
    wps.description = format_description(r['fields'])
    wps.fix = '2d'
    gpx.waypoints.append(wps)

print(gpx.to_xml()[0:1100] + '\n...')

<?xml version="1.0" encoding="UTF-8"?>
<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd" version="1.1" creator="Steve Kasica, kasica@alumni.ubc.ca">
  <metadata>
    <name>Vancouver Public Washrooms</name>
    <desc>Public washrooms in Vancouver</desc>
  </metadata>
  <wpt lat="49.2758809677686" lon="-123.024071960319">
    <time>2020-10-15T10:31:50.455000Z</time>
    <name>Adanac Park Washroom</name>
    <desc>A public toilet in park, open from dawn to dusk with no wheelchair access</desc>
    <src>City of Vancouver</src>
    <sym>Restroom</sym>
    <type>Public washroom</type>
  </wpt>
  <wpt lat="49.2452339943975" lon="-123.175460949361">
    <time>2020-10-15T10:31:50.455000Z</time>
    <name>Balaclava Park Washroom</name>
    <desc>A public toilet in park, open from dawn to dusk with no wheelchair access</desc>
    <src>City of Vancouve

## Export data
Finally, I'll write this data to disk as a GPX file.

In [6]:
with open('vancouver-public-washrooms.gpx', 'w') as f:
    xml = gpx.to_xml()
    f.write(xml)